In [1]:
import sys
sys.path.append('/home/alexj/hgcal-dev')

In [2]:
from hgcal_dev.visualization.hgcal import Event
import numpy as np
import pandas as pd
import torch
from hgcal_dev.utils.experiment import Experiment
from pathlib import Path
from hgcal_dev.clustering.meanshift import MeanShift
from tqdm import tqdm

In [3]:
run_path = Path('/home/alexj/outputs/wandb/run-2tp2l5gx')
experiment = Experiment(run_path)

ERROR:wandb.jupyter:Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: alexschuy (use `wandb login --relogin` to force relogin)


In [4]:
def get_events(split, n=-1):
    if split == 'train':
        dataloader = experiment.datamodule.train_dataloader()
    elif split == 'val':
        dataloader = experiment.datamodule.val_dataloader()
    elif split == 'test':
        dataloader = experiment.datamodule.test_dataloader()
    if n == -1:
        input_paths = dataloader.dataset.events
    else:
        input_paths = dataloader.dataset.events[:n]
    events = []
    for input_path in tqdm(input_paths):
        pred_dir = experiment.run_prediction_dir / split
        event_name = input_path.stem
        pred_path = pred_dir / f'{event_name}.npz'
        events.append(Event(input_path, pred_path=pred_path, task='semantic'))
    return events

In [5]:
events = get_events('train', n=100)

100%|██████████| 100/100 [00:00<00:00, 287.58it/s]


In [11]:
np.unique(events[0].pred_class_labels, return_counts=True)

(array([0, 1, 2]), array([19499,    45,  2202]))

In [13]:
np.unique(events[0].input_event['labels'], return_counts=True)

(array([0, 1, 2]), array([19492,   325,  1929]))

In [15]:
(events[0].pred_class_labels == events[0].input_event['labels']).sum() / events[0].pred_class_labels.shape

array([0.96932769])